In [ ]:
!wget https://files.webis.de/software/pyterrier-plugins/custom-terrier-token-processing-1.0-SNAPSHOT-jar-with-dependencies.jar -O /root/.pyterrier/custom-terrier-token-processing-0.0.1.jar


In [ ]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
import pandas as pd
from tqdm import tqdm

ensure_pyterrier_is_loaded()
tira = Client()

pd.set_option('display.max_colwidth', 0)

if not pt.started():
    pt.init(boot_packages=['mam10eks:custom-terrier-token-processing:0.0.1'])
    from jnius import autoclass

In [ ]:
stopwords= ["I", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your",
              "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", 
              "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", 
              "theirs", "themselves", "what", "which", "who", "whom", "this", "that", 
              "these", "those", "am", "is", "are", "was", "were", "be", "been", "being",
                "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an",
                  "the", "and", "but", "if", "or", "because", "as", "until", "while", "of",
                    "at", "by", "for", "with", "about", "against", "between", "into", "through", 
                    "during", "before", "after", "above", "below", "to", "from", "up", "down", "in",
                      "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", 
                      "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more",
                        "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", 
                        "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [ ]:
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

In [ ]:
stemmer = pt.TerrierStemmer.porter
stemmer1= pt.TerrierStemmer.weakporter
tokenizer = pt.TerrierTokeniser.english


iter_indexer = pt.IterDictIndexer("./passage_index", overwrite=True, verbose= True, stopwords =stopwords, tokeniser=tokenizer, stemmer =stemmer, meta={'docno': 100, 'text': 4096})

print('start')
indexref = iter_indexer.index(tqdm(pt_dataset.get_corpus_iter()))
print('end')

In [ ]:
dph_qe = pt.BatchRetrieve(indexref, wmodel="DPH", controls={"qe":"on", "qemodel" : "KLCorrect"},)

In [ ]:
pt_dataset.get_topics('text').head(3)

print('Now we do the retrieval...')
run = dph_qe(pt_dataset.get_topics('text'))

print('Done. Here are the first 10 entries of the run')
run.head(10)

persist_and_normalize_run(run, system_name='altered index 1', default_output='../runs/anotherExperiment')